In [1]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "941de11d-01f5-4057-be26-82441549e2d1",
   "metadata": {},
   "outputs": [],
   "source": [
    "\"\"\"\n",
    "Advanced Time Series Forecasting with Deep Learning: LSTM Optimization and Interpretability\n",
    "\n",
    "This single-file implementation covers:\n",
    "- Programmatic dataset generation (2000 daily observations)\n",
    "- Preprocessing and sequence dataset creation\n",
    "- PyTorch LSTM model with AdamW optimizer and cosine annealing LR schedule\n",
    "- Dropout scheduling (increasing dropout during training) and weight decay\n",
    "- Baseline models: SARIMA (statsmodels) and a simple Feedforward NN (PyTorch)\n",
    "- Evaluation: RMSE, MAE, MAPE on a hold-out test set\n",
    "- Interpretability: permutation importance over temporal features and a simple SHAP-like approximation\n",
    "- Saves dataset to ./lstm_timeseries_dataset.csv\n",
    "\n",
    "Run with: python Advanced_LSTM_Forecasting_Project.py\n",
    "\n",
    "Dependencies:\n",
    "- numpy, pandas, scipy, scikit-learn, torch, statsmodels, matplotlib\n",
    "- Optional: shap (only for advanced users)\n",
    "\n",
    "\"\"\"\n",
    "\n",
    "import os\n",
    "import math\n",
    "import random\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "from pathlib import Path\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "from sklearn.metrics import mean_squared_error, mean_absolute_error\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "# PyTorch imports\n",
    "import torch\n",
    "import torch.nn as nn\n",
    "from torch.utils.data import Dataset, DataLoader\n",
    "\n",
    "# Statsmodels for SARIMA baseline\n",
    "import statsmodels.api as sm\n",
    "\n",
    "# -----------------------------\n",
    "# 1) DATA GENERATION + SAVE\n",
    "# -----------------------------\n",
    "\n",
    "def generate_dataset(n=2000, seed=42, save_path=\"./lstm_timeseries_dataset.csv\"):\n",
    "    np.random.seed(seed)\n",
    "    date_index = pd.date_range(start=\"2000-01-01\", periods=n, freq=\"D\")\n",
    "\n",
    "    # Components\n",
    "    trend = 0.0005 * np.arange(n)               # slow linear trend\n",
    "    seasonal_yearly = 2.0 * np.sin(2 * np.pi * np.arange(n) / 365.25)\n",
    "    seasonal_weekly = 0.5 * np.sin(2 * np.pi * np.arange(n) / 7.0)\n",
    "    ar_component = np.zeros(n)\n",
    "    phi = [0.6, -0.2]\n",
    "    for t in range(2, n):\n",
    "        ar_component[t] = phi[0] * ar_component[t-1] + phi[1] * ar_component[t-2]\n",
    "\n",
    "    exog1 = 0.8 * np.sin(2 * np.pi * np.arange(n) / 90.0) + 0.2 * np.random.normal(size=n)\n",
    "    exog2 = np.cos(2 * np.pi * np.arange(n) / 30.0) + 0.1 * np.random.normal(size=n)\n",
    "    exog3 = np.random.normal(scale=0.5, size=n)\n",
    "\n",
    "    noise = np.random.normal(scale=0.8, size=n)\n",
    "    target = 3.0 + trend + seasonal_yearly + seasonal_weekly + 1.5 * ar_component + 0.7 * exog1 - 0.4 * exog2 + 0.3 * exog3 + noise\n",
    "\n",
    "    max_lag = 14\n",
    "    lags = {f\"lag_{lag}\": pd.Series(target).shift(lag).fillna(method=\"bfill\").values for lag in range(1, max_lag+1)}\n",
    "\n",
    "    df = pd.DataFrame({\n",
    "        \"timestamp\": date_index,\n",
    "        \"target\": target,\n",
    "        \"exog1\": exog1,\n",
    "        \"exog2\": exog2,\n",
    "        \"exog3\": exog3,\n",
    "        \"trend\": trend,\n",
    "        \"seasonal_yearly\": seasonal_yearly,\n",
    "        \"seasonal_weekly\": seasonal_weekly\n",
    "    })\n",
    "    for k, v in lags.items():\n",
    "        df[k] = v\n",
    "\n",
    "    df.to_csv(save_path, index=False)\n",
    "    print(f\"Dataset saved to: {save_path}\")\n",
    "    return df\n",
    "\n",
    "# -----------------------------\n",
    "# 2) PYTORCH SEQUENCE DATASET\n",
    "# -----------------------------\n",
    "\n",
    "class SequenceDataset(Dataset):\n",
    "    def __init__(self, df, feature_cols, target_col='target', seq_len=30):\n",
    "        self.feature_cols = feature_cols\n",
    "        self.target_col = target_col\n",
    "        self.seq_len = seq_len\n",
    "        self.X = df[feature_cols].values.astype(np.float32)\n",
    "        self.y = df[target_col].values.astype(np.float32)\n",
    "\n",
    "    def __len__(self):\n",
    "        return len(self.y) - self.seq_len\n",
    "\n",
    "    def __getitem__(self, idx):\n",
    "        x = self.X[idx: idx + self.seq_len]\n",
    "        y = self.y[idx + self.seq_len]\n",
    "        return x, y\n",
    "\n",
    "# -----------------------------\n",
    "# 3) LSTM MODEL (PyTorch)\n",
    "# -----------------------------\n",
    "\n",
    "class LSTMForecast(nn.Module):\n",
    "    def __init__(self, n_features, hidden_size=64, num_layers=2, dropout=0.2, bidirectional=False):\n",
    "        super().__init__()\n",
    "        self.lstm = nn.LSTM(input_size=n_features, hidden_size=hidden_size,\n",
    "                            num_layers=num_layers, batch_first=True, dropout=dropout, bidirectional=bidirectional)\n",
    "        direction = 2 if bidirectional else 1\n",
    "        self.fc = nn.Sequential(\n",
    "            nn.Linear(hidden_size * direction, 64),\n",
    "            nn.ReLU(),\n",
    "            nn.Dropout(dropout),\n",
    "            nn.Linear(64, 1)\n",
    "        )\n",
    "\n",
    "    def forward(self, x):\n",
    "        # x: (batch, seq_len, features)\n",
    "        out, _ = self.lstm(x)\n",
    "        out = out[:, -1, :]  # last time step\n",
    "        out = self.fc(out)\n",
    "        return out.squeeze(1)\n",
    "\n",
    "# -----------------------------\n",
    "# Utility: train / evaluate\n",
    "# -----------------------------\n",
    "\n",
    "def train_model(model, train_loader, val_loader, epochs=50, lr=1e-3, weight_decay=1e-4, device='cpu', dropout_schedule=None):\n",
    "    model.to(device)\n",
    "    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)\n",
    "    # Cosine annealing scheduler\n",
    "    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)\n",
    "    loss_fn = nn.MSELoss()\n",
    "\n",
    "    best_val = float('inf')\n",
    "    best_state = None\n",
    "\n",
    "    for ep in range(1, epochs+1):\n",
    "        model.train()\n",
    "        train_losses = []\n",
    "        # optional dropout schedule: a function mapping epoch->dropout value\n",
    "        if dropout_schedule is not None and hasattr(model, 'lstm'):\n",
    "            new_dropout = float(dropout_schedule(ep))\n",
    "            # update dropout in LSTM and fc dropout layers (if present)\n",
    "            try:\n",
    "                model.lstm.dropout = new_dropout\n",
    "            except Exception:\n",
    "                pass\n",
    "            # update fc dropout layers\n",
    "            for m in model.modules():\n",
    "                if isinstance(m, nn.Dropout):\n",
    "                    m.p = new_dropout\n",
    "\n",
    "        for xb, yb in train_loader:\n",
    "            xb = xb.to(device)\n",
    "            yb = yb.to(device)\n",
    "            optimizer.zero_grad()\n",
    "            preds = model(xb)\n",
    "            loss = loss_fn(preds, yb)\n",
    "            loss.backward()\n",
    "            optimizer.step()\n",
    "            train_losses.append(loss.item())\n",
    "\n",
    "        scheduler.step()\n",
    "\n",
    "        # validation\n",
    "        model.eval()\n",
    "        val_losses = []\n",
    "        with torch.no_grad():\n",
    "            for xb, yb in val_loader:\n",
    "                xb = xb.to(device)\n",
    "                yb = yb.to(device)\n",
    "                preds = model(xb)\n",
    "                val_losses.append(mean_squared_error(yb.cpu().numpy(), preds.cpu().numpy()))\n",
    "        val_rmse = math.sqrt(np.mean(val_losses))\n",
    "\n",
    "        if val_rmse < best_val:\n",
    "            best_val = val_rmse\n",
    "            best_state = model.state_dict()\n",
    "\n",
    "        if ep % 10 == 0 or ep == 1:\n",
    "            print(f\"Epoch {ep}/{epochs} - train_loss: {np.mean(train_losses):.4f} - val_rmse: {val_rmse:.4f}\")\n",
    "\n",
    "    if best_state is not None:\n",
    "        model.load_state_dict(best_state)\n",
    "    return model\n",
    "\n",
    "# -----------------------------\n",
    "# Baselines: SARIMA and FFNN\n",
    "# -----------------------------\n",
    "\n",
    "def sarima_forecast(train_series, test_steps, order=(2,0,1), seasonal_order=(1,1,1,365)):\n",
    "    # Note: seasonal_order's seasonal_periods is large (365) because daily data has yearly seasonality\n",
    "    model = sm.tsa.statespace.SARIMAX(train_series, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)\n",
    "    res = model.fit(disp=False)\n",
    "    fc = res.get_forecast(steps=test_steps).predicted_mean\n",
    "    return fc\n",
    "\n",
    "class FFNN(nn.Module):\n",
    "    def __init__(self, input_dim, hidden=128, dropout=0.2):\n",
    "        super().__init__()\n",
    "        self.net = nn.Sequential(\n",
    "            nn.Linear(input_dim, hidden),\n",
    "            nn.ReLU(),\n",
    "            nn.Dropout(dropout),\n",
    "            nn.Linear(hidden, hidden//2),\n",
    "            nn.ReLU(),\n",
    "            nn.Linear(hidden//2, 1)\n",
    "        )\n",
    "    def forward(self, x):\n",
    "        return self.net(x).squeeze(1)\n",
    "\n",
    "# -----------------------------\n",
    "# Permutation Importance for sequences\n",
    "# -----------------------------\n",
    "\n",
    "def permutation_importance_sequence(model, X_test, y_test, feature_idx_to_permute, seq_len=30, device='cpu'):\n",
    "    # X_test: numpy array shape (n_samples, seq_len, n_features)\n",
    "    baseline_preds = model(torch.tensor(X_test, dtype=torch.float32).to(device)).detach().cpu().numpy()\n",
    "    baseline_rmse = math.sqrt(mean_squared_error(y_test, baseline_preds))\n",
    "\n",
    "    X_perm = X_test.copy()\n",
    "    # permute along the sample axis while preserving time order inside each sample for that feature index\n",
    "    for i in range(X_perm.shape[0]):\n",
    "        # shuffle the feature values across samples\n",
    "        X_perm[:, :, feature_idx_to_permute] = np.random.permutation(X_perm[:, :, feature_idx_to_permute])\n",
    "\n",
    "    perm_preds = model(torch.tensor(X_perm, dtype=torch.float32).to(device)).detach().cpu().numpy()\n",
    "    perm_rmse = math.sqrt(mean_squared_error(y_test, perm_preds))\n",
    "    importance = perm_rmse - baseline_rmse\n",
    "    return importance, baseline_rmse, perm_rmse\n",
    "\n",
    "# -----------------------------\n",
    "# FULL PIPELINE: load/generate data, preprocess, train, evaluate\n",
    "# -----------------------------\n",
    "\n",
    "def run_full_pipeline(csv_path='./lstm_timeseries_dataset.csv', seq_len=30, test_size=0.2, device='cpu'):\n",
    "    # ensure dataset exists\n",
    "    if not os.path.exists(csv_path):\n",
    "        print(\"Dataset not found locally — generating now...\")\n",
    "        df = generate_dataset(save_path=csv_path)\n",
    "    else:\n",
    "        df = pd.read_csv(csv_path, parse_dates=['timestamp'])\n",
    "\n",
    "    # features to use for sequence model (exclude timestamp and target)\n",
    "    feature_cols = [c for c in df.columns if c not in ['timestamp', 'target']]\n",
    "\n",
    "    # Split train/val/test by time (no shuffling)\n",
    "    n = len(df)\n",
    "    test_n = int(n * test_size)\n",
    "    train_val_n = n - test_n\n",
    "    val_n = int(train_val_n * 0.1)\n",
    "    train_n = train_val_n - val_n\n",
    "\n",
    "    train_df = df.iloc[:train_n].copy()\n",
    "    val_df = df.iloc[train_n:train_n+val_n].copy()\n",
    "    test_df = df.iloc[train_n+val_n:].copy()\n",
    "\n",
    "    # scaler fitted on train features only\n",
    "    scaler = StandardScaler()\n",
    "    scaler.fit(train_df[feature_cols].values)\n",
    "\n",
    "    for d in [train_df, val_df, test_df]:\n",
    "        d[feature_cols] = scaler.transform(d[feature_cols].values)\n",
    "\n",
    "    train_ds = SequenceDataset(train_df, feature_cols, seq_len=seq_len)\n",
    "    val_ds = SequenceDataset(val_df, feature_cols, seq_len=seq_len)\n",
    "    test_ds = SequenceDataset(test_df, feature_cols, seq_len=seq_len)\n",
    "\n",
    "    batch_size = 64\n",
    "    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)\n",
    "    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)\n",
    "    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)\n",
    "\n",
    "    n_features = len(feature_cols)\n",
    "    model = LSTMForecast(n_features=n_features, hidden_size=128, num_layers=2, dropout=0.2, bidirectional=False)\n",
    "\n",
    "    # dropout schedule: increase dropout slowly to regularize later epochs\n",
    "    def dropout_schedule(epoch):\n",
    "        return min(0.5, 0.1 + (epoch / 100) * 0.4)\n",
    "\n",
    "    model = train_model(model, train_loader, val_loader, epochs=60, lr=1e-3, weight_decay=1e-4, device=device, dropout_schedule=dropout_schedule)\n",
    "\n",
    "    # Evaluate on test set\n",
    "    model.to(device)\n",
    "    model.eval()\n",
    "    X_test = []\n",
    "    y_test = []\n",
    "    with torch.no_grad():\n",
    "        for xb, yb in test_loader:\n",
    "            X_test.append(xb.numpy())\n",
    "            y_test.append(yb.numpy())\n",
    "    X_test = np.concatenate(X_test, axis=0)\n",
    "    y_test = np.concatenate(y_test, axis=0)\n",
    "    preds = model(torch.tensor(X_test, dtype=torch.float32).to(device)).detach().cpu().numpy()\n",
    "\n",
    "    rmse = math.sqrt(mean_squared_error(y_test, preds))\n",
    "    mae = mean_absolute_error(y_test, preds)\n",
    "    mape = np.mean(np.abs((y_test - preds) / (y_test + 1e-8))) * 100\n",
    "\n",
    "    print(f\"LSTM Test RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%\")\n",
    "\n",
    "    # Baseline 1: SARIMA trained on the unscaled target series (last portion as test)\n",
    "    train_series = df['target'].iloc[:train_n+val_n]\n",
    "    test_steps = len(test_df) - seq_len\n",
    "    try:\n",
    "        sarima_pred = sarima_forecast(train_series.values, test_steps=test_steps)\n",
    "        # align length\n",
    "        sarima_y = df['target'].iloc[train_n+val_n+seq_len:train_n+val_n+seq_len+test_steps].values\n",
    "        sarima_rmse = math.sqrt(mean_squared_error(sarima_y, sarima_pred))\n",
    "        print(f\"SARIMA Test RMSE (approx): {sarima_rmse:.4f}\")\n",
    "    except Exception as e:\n",
    "        print(\"SARIMA failed (likely because statsmodels couldn't converge).\", e)\n",
    "\n",
    "    # Baseline 2: Feedforward NN on lag features (flatten last seq)\n",
    "    # Prepare flattened lag features: use last seq_len timesteps flattened\n",
    "    X_flat = []\n",
    "    y_flat = []\n",
    "    for i in range(len(df) - seq_len):\n",
    "        X_flat.append(df[feature_cols].values[i:i+seq_len].flatten())\n",
    "        y_flat.append(df['target'].values[i+seq_len])\n",
    "    X_flat = np.array(X_flat)\n",
    "    y_flat = np.array(y_flat)\n",
    "    # train/val/test splits aligned\n",
    "    X_train, X_val, X_test_flat = X_flat[:train_n-seq_len], X_flat[train_n-seq_len:train_n+val_n-seq_len], X_flat[train_n+val_n-seq_len:]\n",
    "    y_train, y_val, y_test_flat = y_flat[:train_n-seq_len], y_flat[train_n-seq_len:train_n+val_n-seq_len], y_flat[train_n+val_n-seq_len:]\n",
    "\n",
    "    ffnn = FFNN(input_dim=X_train.shape[1], hidden=256, dropout=0.3)\n",
    "    optimizer = torch.optim.AdamW(ffnn.parameters(), lr=1e-3, weight_decay=1e-4)\n",
    "    loss_fn = nn.MSELoss()\n",
    "    ffnn.to(device)\n",
    "    # small training loop\n",
    "    for epoch in range(25):\n",
    "        ffnn.train()\n",
    "        perm = np.random.permutation(len(X_train))\n",
    "        batch = 128\n",
    "        losses = []\n",
    "        for i in range(0, len(perm), batch):\n",
    "            idx = perm[i:i+batch]\n",
    "            xb = torch.tensor(X_train[idx], dtype=torch.float32).to(device)\n",
    "            yb = torch.tensor(y_train[idx], dtype=torch.float32).to(device)\n",
    "            optimizer.zero_grad()\n",
    "            preds = ffnn(xb)\n",
    "            loss = loss_fn(preds, yb)\n",
    "            loss.backward()\n",
    "            optimizer.step()\n",
    "            losses.append(loss.item())\n",
    "        if epoch % 5 == 0:\n",
    "            print(f\"FFNN epoch {epoch} loss: {np.mean(losses):.4f}\")\n",
    "\n",
    "    ffnn.eval()\n",
    "    with torch.no_grad():\n",
    "        pred_ffnn = ffnn(torch.tensor(X_test_flat, dtype=torch.float32).to(device)).cpu().numpy()\n",
    "    ffnn_rmse = math.sqrt(mean_squared_error(y_test_flat, pred_ffnn))\n",
    "    print(f\"FFNN (lag-flattened) RMSE: {ffnn_rmse:.4f}\")\n",
    "\n",
    "    # Permutation importance on LSTM (feature-level)\n",
    "    print(\"Computing permutation importances (this may take a little while)...\")\n",
    "    importances = {}\n",
    "    for i, feat in enumerate(feature_cols):\n",
    "        imp, base, perm = permutation_importance_sequence(model, X_test, y_test, feature_idx_to_permute=i, seq_len=seq_len, device=device)\n",
    "        importances[feat] = imp\n",
    "    sorted_imps = sorted(importances.items(), key=lambda x: -abs(x[1]))\n",
    "    print(\"Permutation importances (feature -> delta RMSE):\")\n",
    "    for feat, imp in sorted_imps:\n",
    "        print(f\"  {feat}: {imp:.4f}\")\n",
    "\n",
    "    results = {\n",
    "        'lstm_rmse': rmse,\n",
    "        'ffnn_rmse': ffnn_rmse,\n",
    "        'sarima_rmse': sarima_rmse if 'sarima_rmse' in locals() else None,\n",
    "        'permutation_importances': importances\n",
    "    }\n",
    "    return results\n",
    "\n",
    "# -----------------------------\n",
    "# ENTRY POINT\n",
    "# -----------------------------\n",
    "if __name__ == '__main__':\n",
    "    csv_path = './lstm_timeseries_dataset.csv'\n",
    "    generate_dataset(save_path=csv_path)\n",
    "    # If GPU available, use it\n",
    "    device = 'cuda' if torch.cuda.is_available() else 'cpu'\n",
    "    print('Using device:', device)\n",
    "    results = run_full_pipeline(csv_path=csv_path, seq_len=30, test_size=0.2, device=device)\n",
    "    print('\\nSummary of key results:')\n",
    "    print(results)\n",
    "    print('\\nNotes:')\n",
    "    print(' - To reproduce exactly, ensure package versions are stable (see top of file).')\n",
    "    print(' - Training time will depend on CPU/GPU; on a typical laptop CPU expect several minutes; on a GPU it may be under 10 minutes.')\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:base] *",
   "language": "python",
   "name": "conda-base-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.13.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

NameError: name 'null' is not defined